## FII_SMART Multi-evidence Natural Language Inference for Clinical Trial Data


In [1]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sun Feb 12 14:07:40 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   61C    P0    25W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
import sys
import os

DEBUG1=False
DEBUG2=False

PREFIXG="/content/"
PREFIX="/content/"

TRAIN_DATA=PREFIXG+"train.csv"
TEST_DATA=PREFIXG+"dev.csv"
TEST_DATA2=PREFIXG+"test.csv"

Clean files for new run

In [16]:
from google.colab import files

!ls
!rm test.*
!rm train.*
!rm dev.*
!rm -rf CT_data
!rm CT_data.zip
!rm dataset.zip

CT_data  sample_data
rm: cannot remove 'test.*': No such file or directory
rm: cannot remove 'train.*': No such file or directory
rm: cannot remove 'dev.*': No such file or directory
rm: cannot remove 'CT_data.zip': No such file or directory
rm: cannot remove 'dataset.zip': No such file or directory


In [13]:
!pip install transformers
!wget https://github.com/volosincu/FII_Smart__Semeval2023/raw/main/dataset.zip
!unzip /content/dataset.zip
!unzip /content/CT_data.zip

--2023-02-12 14:12:11--  https://github.com/volosincu/FII_Smart__Semeval2023/raw/main/dataset.zip
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/volosincu/FII_Smart__Semeval2023/main/dataset.zip [following]
--2023-02-12 14:12:12--  https://raw.githubusercontent.com/volosincu/FII_Smart__Semeval2023/main/dataset.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2479891 (2.4M) [application/zip]
Saving to: ‘dataset.zip’

dataset.zip         100%[===================>]   2.36M  11.0MB/s    in 0.2s    

2023-02-12 14:12:12 (11.0 MB/s) - ‘dataset.zip’ saved [2479891/2479891]

Archive:  /content/datas

In [7]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
print(stopwords.words('english'))

STOP_WORDS_RO = ['e.g.', '•','â•','mm¬', 'mm¬¨','trial','study','patient', 'patients', 'candidate', 'candidates', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'a', 'an', 'the', 'here', 'there', 'why', 'how', 'so', 's', 't', 'can', 'will', 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ma', ]

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [8]:

punctuation = ['„', '”', '(', ')', ';', '{', '}','[',']', '/', '\\',"\"", "'", ','] 
def normalizeChunk(chunk):
  _word = chunk.lower()
  word = None
  withSeparator = False;
  if _word.startswith('('):
    word = _word[1:]
  elif _word.endswith(')'):
    word = _word[:len(_word)-1]
  # elif _word.endswith(','):
  #   word = _word[:len(_word)-1]
  # elif _word.endswith('.'):
  #   # withSeparator = True
  #   word = _word[:len(_word)-1]
  elif _word.endswith(':'):
    word = _word[:len(_word)-1]
  elif _word.endswith(';'):
    word = _word[:len(_word)-1]
  elif _word.startswith('„'):
    word = _word[1:]
  elif _word.endswith('”'):
    word = _word[:len(_word)-1]
  elif _word.startswith('{'):
    word = _word[1:]
  elif _word.startswith('['):
    word = _word[1:]
  elif _word.startswith("\'"):
    word = _word[1:]
  elif _word.endswith('\''):
    word = _word[:len(_word)-1]
  elif _word.endswith('}'):
    word = _word[:len(_word)-1]
  elif _word.endswith(']'):
    word = _word[:len(_word)-1]
  else:
    word = _word

  concats=[]
  for sign in punctuation:
    if(word.__contains__(sign)):
      c = word.split(sign)
      for w in c:
        sw = normalizeChunk(w)
        for w1 in sw:
          if(not concats.__contains__(sw)):
            concats.append(w1)
  if(len(concats) == 0):
    concats.append(word)
    if(withSeparator):
      concats.append("<START>")
  return list(dict.fromkeys(concats))

normalizeChunk("casa.„Toata")

['casa.', 'toata']

In [9]:
def normalizeDocument(text, removeStopWords):
  NORMALIZED_CORPUS = []
  for _word in text.split():
    if(_word.__contains__('>=')):
      _word = _word.replace(">=", "greater than or equal ")
      #print(_word)
    if(_word.__contains__('=<')):
      _word = _word.replace("=<", "smaller than or equal ")
      #print(_word)
    if(_word.__contains__('<')):
      _word = _word.replace("<", "smaller than ")
      #print(_word)
    if(_word.__contains__('>')):
      _word = _word.replace(">", "greater than ")
      #print(_word)
    normalized = normalizeChunk(_word)
    for word in normalized:
      if(removeStopWords):
        if(not STOP_WORDS_RO.__contains__(word)):
          NORMALIZED_CORPUS.append(word)
      else:
        NORMALIZED_CORPUS.append(word)
  return " ".join(NORMALIZED_CORPUS)

normalizat = normalizeDocument("Renal function: Creatinine =< 2.0 mg/dl or a creatinine clearance >= 50 mg/min", True)
normalizat

'renal function creatinine smaller than or equal  2.0 mg dl or creatinine clearance greater than or equal  50 mg min'

In [10]:
import uuid
import json
from transformers import BertTokenizer, TFBertModel
import matplotlib.pyplot as plt
import tensorflow as tf

import numpy as np
import pandas as pd


from transformers import AutoTokenizer, AutoModel


from datetime import datetime
current_dateTime = datetime.now()

In [11]:
def stamp_time():
    return "".join("".join("".join(str(current_dateTime).split(".")[0].split(":")).split("-")).split(" "))


In [12]:
OVF=0
OVF_V=[]

In [13]:
VECTOR_MAX=130

EPOCHS = 2
LAYER_1_NODES=256
LAYER_2_NODES=2
LAYER_1_ACTIVATION='relu'
LAYER_2_ACTIVATION='softmax'
LOSS_FN='sparse_categorical_crossentropy'
#LOSS_FN='binary_crossentropy'
METRIC='accuracy'
FIT_BATCH_SIZE=16

In [14]:
from transformers import BertTokenizer, BertModel, BertForMaskedLM

model_name = 'bert-large-uncased'
# model_name = 'giacomomiolo/biobert_reupload'
# model_name = 'dmis-lab/biobert-base-cased-v1.1'
# model_name = 'bert-base-multilingual-cased'
# model_name = "allenai/scibert_scivocab_uncased"

# tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer = BertTokenizer.from_pretrained(model_name)

## Data collection utils


In [15]:
def encode_sentence(s):
   tokens = list(tokenizer.tokenize(s))
   tokens.append('[SEP]')
   return tokenizer.convert_tokens_to_ids(tokens)


def collectEvidenceData(evidenceSection, evidenceIndexes, clinicalTrialId, evidenceSectionId, DEBUG2):
    evidences = []
    for evIndex in evidenceIndexes:
        try:
            i = int(evIndex)
            if DEBUG2:
                print(evidenceSection[i])
            norm_evid = normalizeDocument(evidenceSection[i].strip(), True)
            evidences.append(norm_evid)
        except:
            print(evidenceIndexes)
            print('\n!!! Failed to collect evidence on index, trialId, section: ' + str(i) +' '+ clinicalTrialId, evidenceSectionId)
    return evidences


def parseEvidenceSingle(hypotesis, clinicalTrialId, evidenceSectionId, primaryEvidenceIndexes, PREFIX, DEBUG1, DEBUG2):
    evidences = []
    if DEBUG1:
        print("Parse evidence Single: " + clinicalTrialId + " " + evidenceSectionId)
    CT_PATH = PREFIX+'CT_data/'+clinicalTrialId+'.json'
    try:
        with open(CT_PATH) as json_file:
            dataCT = json.load(json_file)
            if DEBUG2:
                print(primaryEvidenceIndexes)
        evidences = collectEvidenceData(dataCT[evidenceSectionId], primaryEvidenceIndexes, clinicalTrialId, evidenceSectionId, False)
        if DEBUG2:
            print(evidences)
        tok = encode_sentence("[CLS] "+hypotesis+" [SEP] " + " ".join(evidences))
        # print("[CLS] "+hypotesis+" [SEP] " + " ".join(evidences))
        # print("       aa          ")
        # if (len(tok)>300):
        #     # OVF=OVF+1
        #     OVF_V.append(len(tok))
        #     print("!!!!!!!!!!!!!!!!")
        #     print(hypotesis)
        #     print(clinicalTrialId)

    except Exception as err:
        print('\n!!! Error for file: ' + clinicalTrialId)
        print(err)
    if DEBUG2:
        print("\n")
    return evidences

In [16]:
def splitEvidence(acc, tmp, hypotesis, evidences, evIndex, DEBUG):
  if (len(evidences)==0):
    return acc
  if (evIndex>len(evidences)-1):
    acc.append(tmp)
    # print("----------1-----------EXIT split evidenta ---------------------")
    return acc;
  # print(evidences)
  # print(evIndex)
  hypTmpl = "[CLS] "+hypotesis+" [SEP] "
  tok = encode_sentence(hypTmpl + tmp + evidences[evIndex]+" ")
  if(len(tok)<VECTOR_MAX):
    tmp = tmp + evidences[evIndex]+" "
    splitEvidence(acc, tmp, hypotesis, evidences, evIndex+1, DEBUG)
  else:
    acc.append(tmp)
    tmp=""
    tok = encode_sentence(hypTmpl + tmp + evidences[evIndex]+" ")
    if(len(tok)<VECTOR_MAX):
      if (evIndex<len(evidences)):
        tmp = tmp + evidences[evIndex]
        splitEvidence(acc, tmp, hypotesis, evidences, evIndex+1, DEBUG)
      else:
        # print("----------2-----------EXIT split evidenta ---------------------")
        acc.append(evidences[evIndex])
        return acc
    else:
      if (DEBUG):
        print("!!!!!!!! hypotesis + evidence > " + str(VECTOR_MAX) + "!!!!!!!!!!")
      return acc
  return acc


In [17]:
STM = pd.read_csv(TRAIN_DATA) #, nrows=40
print(STM.head(5))
print(STM.shape)

# exit()
_EVIDENCES=[]
_HYPOTESIS=[]
Y_LABELS=np.array([], dtype="int32")
CT_index = -1
for id in STM['Id']:
    evidences = []
    CT_index = CT_index+1
    CTid = STM['Primary_id'][CT_index]
    hypotesis = normalizeDocument(STM['Statement'][CT_index], True)
    hypotesisLen = len(hypotesis.split(' '))
    if(hypotesisLen > 90):
      print('%%%% ')
    if (STM['Type'][CT_index] == "Single"):
      primaryEvidenceIndexes = STM['Primary_evidence_index'][CT_index].split(',')
      evidences = parseEvidenceSingle(hypotesis, CTid, STM['Section_id'][CT_index], primaryEvidenceIndexes, PREFIX, DEBUG1, DEBUG2)
      ax = splitEvidence([], "", hypotesis, evidences, 0, DEBUG2)
      if (CT_index < 17):
        print(hypotesis)
      for evidenceFragment in ax:
        # parse only evidences those that have a minimum of words 
        if(hypotesisLen+len(evidenceFragment) > hypotesisLen + 10):
          _EVIDENCES.append(evidenceFragment)
          _HYPOTESIS.append(hypotesis)
          if(STM['Label'][CT_index] == 'Entailment'):
            Y_LABELS = np.append(Y_LABELS, [1], axis=0)
          else:
            Y_LABELS = np.append(Y_LABELS, [0], axis=0)
          if (CT_index < 17):
            print("     - "+ evidenceFragment)
      
      # if (CT_index == 3):
      #     exit()
      if DEBUG1:
          print('Analyzed CT entry: ' + str(CT_index) + "  -  "+ CTid+"\n")
    if (STM['Type'][CT_index] == "Comparison"):
      CTidSecondary = STM['Secondary_id'][CT_index]
      primaryEvidenceIndexes = STM['Primary_evidence_index'][CT_index].split(',')
      secondaryEvidenceIndexes = STM['Secondary_evidence_index'][CT_index].split(',')
      premisesPrimary = parseEvidenceSingle(hypotesis, CTid, STM['Section_id'][CT_index], primaryEvidenceIndexes, PREFIX, DEBUG1, DEBUG2)
      premisesSecondary = parseEvidenceSingle(hypotesis, CTidSecondary, STM['Section_id'][CT_index], secondaryEvidenceIndexes, PREFIX, DEBUG1, DEBUG2)
      premisesIndexed = splitEvidence([], "", hypotesis, premisesPrimary + premisesSecondary, 0, DEBUG2)

      if (CT_index < 17):
        print(hypotesis)
      for evidenceFragment in premisesIndexed:
        # parse only evidences those that have a minimum of words 
        if(hypotesisLen+len(evidenceFragment) > hypotesisLen + 10):
          _EVIDENCES.append(evidenceFragment)
          _HYPOTESIS.append(hypotesis)
          if(STM['Label'][CT_index] == 'Entailment'):
            Y_LABELS = np.append(Y_LABELS, [1], axis=0)
          else:
            Y_LABELS = np.append(Y_LABELS, [0], axis=0)
        if (CT_index < 17):
          print("     - "+ evidenceFragment)
        
      # if (CT_index == 3):
      #     exit()
      if DEBUG1:
          print('Analyzed CT entry: ' + str(CT_index) + "  -  "+ CTid+"\n")




Y_LABELS

                                     Id        Type      Section_id  \
0  5bc844fc-e852-4270-bfaf-36ea9eface3d  Comparison    Intervention   
1  86b7cb3d-6186-4a04-9aa6-b174ab764eed      Single     Eligibility   
2  dbed5471-c2fc-45b5-b26f-430c9fa37a37  Comparison  Adverse Events   
3  20c35c89-8d23-4be3-b603-ac0ee0f3b4de      Single     Eligibility   
4  f17cb242-419d-4f5d-bfa4-41494ed5ac0e  Comparison    Intervention   

    Primary_id Secondary_id  \
0  NCT01928186  NCT00684983   
1  NCT00662129          NaN   
2  NCT00093145  NCT00703326   
3  NCT01097642          NaN   
4  NCT00852930  NCT02308020   

                                           Statement          Label  \
0  All the primary trial participants do not rece...  Contradiction   
1  Patients with Platelet count over 100,000/mm¬¨...  Contradiction   
2  Heart-related adverse events were recorded in ...     Entailment   
3  Adult Patients with histologic confirmation of...  Contradiction   
4  Laser Therapy is in each coh

array([0, 0, 0, ..., 1, 1, 0])

## Encoding

In [18]:
def encode_data_to_tensors(statements, evidences, tokenizer, dim):
    
    tokStm = [encode_sentence(s)
        for s in statements]

    tokPrem = [encode_sentence(s)
        for s in evidences]
    tokZip = list(zip(tokStm, tokPrem))

    # tokIntersect = [np.where(np.in1d(xt[0], np.intersect1d(xt[0], xt[1])), np.zeros_like(xt[0]), [1])
    #     for xt in tokZip]
    # type_s1 = tf.ragged.constant(tokIntersect);

    tokIntersect = [np.where(np.in1d(xt[1], np.intersect1d(xt[0], xt[1])), np.zeros_like(xt[1]), [1])
        for xt in tokZip]
    type_s2 = tf.ragged.constant(tokIntersect);

    statementsTokenizedTensor = tf.ragged.constant(tokStm)
    evidencesTokenizedTensor = tf.ragged.constant(tokPrem)


    cls = [tokenizer.convert_tokens_to_ids(['[CLS]'])]*statementsTokenizedTensor.shape[0]
    _word_ids = tf.concat([cls, statementsTokenizedTensor, evidencesTokenizedTensor], axis=-1)
    ii = 0
    max = 0
    for ip in _word_ids:
        ii = ii + 1
        if (ip.shape[0] >= max):
            max = ip.shape[0]
            # print(ii)
            # print(max)
    print("Dataset MAX (dim): ")
    print(max)

    # Padding - adaug 0 sau 1 la tensori pt a avea aceiasi dimensiune 
    # paggings [ top bottom ] [left right] - adaug doar la final
    paddings = tf.constant([[0, 0,], [0, dim-max]])
    input_word_ids = tf.pad(_word_ids.to_tensor(), paddings, "CONSTANT", constant_values=0)
    input_mask = tf.ones_like(input_word_ids)

    type_cls = tf.zeros_like(cls)
    type_s1 = tf.zeros_like(statementsTokenizedTensor)
    # type_s2 = tf.ones_like(evidencesTokenizedTensor)
    input_type_ids = tf.pad(tf.concat(
        [type_cls, type_s1, type_s2], axis=-1).to_tensor(), paddings, "CONSTANT", constant_values=0)

    inputs = {
        'input_word_ids': input_word_ids,
        'input_mask': input_mask,
        'input_type_ids': input_type_ids}

    return inputs


## Model declaration

In [19]:
train_input = encode_data_to_tensors(_HYPOTESIS, _EVIDENCES, tokenizer, VECTOR_MAX)
# check shape data 
print('DEBUG train shapes: ')
print(train_input['input_word_ids'].shape)
print(train_input['input_mask'].shape)
print(train_input['input_type_ids'].shape)

Dataset MAX (dim): 
129
DEBUG train shapes: 
(3383, 130)
(3383, 130)
(3383, 130)


In [20]:
def build_model():
    # bert_encoder = TFBertModel.from_pretrained(model_name)
    # bert_encoder = AutoModel.from_pretrained(model_name)
    input_word_ids = tf.keras.Input(shape=(VECTOR_MAX,), dtype=tf.int32, name="input_word_ids")
    input_mask = tf.keras.Input(shape=(VECTOR_MAX,), dtype=tf.int32, name="input_mask")
    input_type_ids = tf.keras.Input(shape=(VECTOR_MAX,), dtype=tf.int32, name="input_type_ids")
    # print(input_word_ids)

    #embedding = bert_encoder([input_word_ids, input_mask, input_type_ids])[0]

    # lstm = tf.keras.layers.LSTM(40, return_sequences=True, return_state=True)(embedding[:,0,:])
    # lstm = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64))(input_word_ids)

    # x = tf.keras.layers.Dense(LAYER_1_NODES, kernel_initializer = 'random_uniform', activation=LAYER_1_ACTIVATION)(embedding[:,0,:])
    x = tf.keras.layers.Dense(512, activation=LAYER_1_ACTIVATION)(input_word_ids)
    x = tf.keras.layers.LeakyReLU(0.9)(x)
    x = tf.keras.layers.Dense(2048, activation=LAYER_1_ACTIVATION)(x)
    x = tf.keras.layers.LeakyReLU(0.9)(x)
    x = tf.keras.layers.Dense(4096, activation=LAYER_1_ACTIVATION)(x)
    x = tf.keras.layers.LeakyReLU(0.9)(x)
    x = tf.keras.layers.Dense(18048, activation=LAYER_1_ACTIVATION)(x)
    x = tf.keras.layers.LeakyReLU(0.9)(x)
    x = tf.keras.layers.Dense(9128, activation=LAYER_1_ACTIVATION)(x)
    x = tf.keras.layers.LeakyReLU(0.9)(x)
    x = tf.keras.layers.Dense(1024, activation=LAYER_1_ACTIVATION)(x)
    x = tf.keras.layers.LeakyReLU(0.9)(x)
    x = tf.keras.layers.Dense(64, activation=LAYER_1_ACTIVATION)(x)
    x = tf.keras.layers.LeakyReLU(0.9)(x)
    output = tf.keras.layers.Dense(2, activation=LAYER_2_ACTIVATION)(x)

    learning_rate = 5e-5


    model = tf.keras.Model(inputs=[input_word_ids], outputs=output)
    model.compile(tf.keras.optimizers.Adam(learning_rate=learning_rate), loss=LOSS_FN, metrics=[METRIC])  # decay=learning_rate/(epochs*0.5)
    # decay=learning_rate/(EPOCHS*0.15)
    return model


# Fit and Run model

In [21]:
model = build_model()
df = pd.DataFrame()
df['Y'] = Y_LABELS


print("dim ipoteze:")
print(len(_HYPOTESIS))
print("dim evidențe:")
print(len(_EVIDENCES))
print("dim Y:")
print(len(Y_LABELS))

model.fit(x=train_input['input_word_ids'], y=Y_LABELS, epochs = 17, verbose = 1, batch_size = FIT_BATCH_SIZE, validation_split = 0.3)

dim ipoteze:
3383
dim evidențe:
3383
dim Y:
3383
Epoch 1/17
148/148 [==============================] - 15s 57ms/step - loss: 111.9804 - accuracy: 0.5004 - val_loss: 51.5037 - val_accuracy: 0.4926
Epoch 2/17
148/148 [==============================] - 8s 55ms/step - loss: 18.1378 - accuracy: 0.5988 - val_loss: 23.7161 - val_accuracy: 0.4768
Epoch 3/17
148/148 [==============================] - 8s 55ms/step - loss: 6.6907 - accuracy: 0.7352 - val_loss: 15.1616 - val_accuracy: 0.4877
Epoch 4/17
148/148 [==============================] - 8s 55ms/step - loss: 3.7258 - accuracy: 0.7990 - val_loss: 12.0182 - val_accuracy: 0.4562
Epoch 5/17
148/148 [==============================] - 8s 56ms/step - loss: 3.4806 - accuracy: 0.8281 - val_loss: 9.9819 - val_accuracy: 0.4640
Epoch 6/17
148/148 [==============================] - 8s 55ms/step - loss: 2.1272 - accuracy: 0.8830 - val_loss: 8.6751 - val_accuracy: 0.4453
Epoch 7/17
148/148 [==============================] - 8s 55ms/step - loss: 1.9350 - a

## Evaluare, test data


In [44]:
TEST_STM = pd.read_csv(TEST_DATA) #, nrows=40
print(TEST_STM.head(5))
print(TEST_STM.shape)


# exit()
TEST_EVIDENCES=[]
TEST_HYPOTESIS=[]
TEST_UUID=[]
TEST_LABEL=[]
TEST_Y_LABELS=np.array([], dtype="int32")
CT_index = -1
for id in TEST_STM['Id']:
    evidences = []
    CT_index = CT_index+1
    CTid = TEST_STM['Primary_id'][CT_index]
    hypotesis = normalizeDocument(TEST_STM['Statement'][CT_index], True)
    hypotesisLen = len(hypotesis.split(' '))
    if (TEST_STM['Type'][CT_index] == "Single"):
      primaryEvidenceIndexes = TEST_STM['Primary_evidence_index'][CT_index].split(',')
      evidences = parseEvidenceSingle(hypotesis, CTid, TEST_STM['Section_id'][CT_index], primaryEvidenceIndexes, PREFIX, DEBUG1, DEBUG2)
      ax = splitEvidence([], "", "[CLS] "+hypotesis+" [SEP] ", evidences, 0, DEBUG2)
      for evidenceFragment in ax:
        # parse only evidences those that have a minimum of words 
        if(hypotesisLen+len(evidenceFragment) > hypotesisLen + 10):
          TEST_EVIDENCES.append(evidenceFragment)
          TEST_HYPOTESIS.append(hypotesis)
          TEST_UUID.append(id)
          TEST_LABEL.append(TEST_STM['Label'][CT_index])
          if(TEST_STM['Label'][CT_index] == 'Entailment'):
            TEST_Y_LABELS = np.append(TEST_Y_LABELS, [1], axis=0)
          else:
            TEST_Y_LABELS = np.append(TEST_Y_LABELS, [0], axis=0)
      if DEBUG1:
          print('Analyzed CT entry: ' + str(CT_index) + "  -  "+ CTid+"\n")
    if (TEST_STM['Type'][CT_index] == "Comparison"):
      CTidSecondary = TEST_STM['Secondary_id'][CT_index]
      primaryEvidenceIndexes = TEST_STM['Primary_evidence_index'][CT_index].split(',')
      secondaryEvidenceIndexes = TEST_STM['Secondary_evidence_index'][CT_index].split(',')
      premisesPrimary = parseEvidenceSingle(hypotesis, CTid, TEST_STM['Section_id'][CT_index], primaryEvidenceIndexes, PREFIX, DEBUG1, DEBUG2)
      premisesSecondary = parseEvidenceSingle(hypotesis, CTidSecondary, TEST_STM['Section_id'][CT_index], secondaryEvidenceIndexes, PREFIX, DEBUG1, DEBUG2)
      premisesIndexed = splitEvidence([], "", hypotesis, premisesPrimary + premisesSecondary, 0, DEBUG2)
      if (CT_index < 17):
        print(hypotesis)
      for evidenceFragment in premisesIndexed:
        # parse only evidences those that have a minimum of words 
        if(hypotesisLen+len(evidenceFragment) > hypotesisLen + 10):
          TEST_EVIDENCES.append(evidenceFragment)
          TEST_HYPOTESIS.append(hypotesis)
          TEST_UUID.append(id)
          TEST_LABEL.append(TEST_STM['Label'][CT_index])
          if(TEST_STM['Label'][CT_index] == 'Entailment'):
            TEST_Y_LABELS = np.append(TEST_Y_LABELS, [1], axis=0)
          else:
            TEST_Y_LABELS = np.append(TEST_Y_LABELS, [0], axis=0)
        if (CT_index < 17):
          print("     - "+ evidenceFragment)
      # print("-------2--------")
      if DEBUG1:
          print('Analyzed CT entry: ' + str(CT_index) + "  -  "+ CTid+"\n")


print("dim ipoteze:")
print(len(TEST_EVIDENCES))
print("dim evidențe:")
print(len(TEST_EVIDENCES))
print("dim Y:")
print(len(TEST_Y_LABELS))

                                     Id        Type      Section_id  \
0  1adc970c-d433-44d0-aa09-d3834986f7a2      Single         Results   
1  6b9162d0-0816-46d4-81af-c60028dcc63b  Comparison     Eligibility   
2  0b6cc8e3-69ee-4a91-b93d-2ad3fddce65f  Comparison  Adverse Events   
3  cc1f712a-2116-4e40-9810-f315e3fa5ff8      Single         Results   
4  904061c0-14fa-4f13-9118-9a41e24fa8eb      Single     Eligibility   

    Primary_id Secondary_id  \
0  NCT00066573          NaN   
1  NCT00425854  NCT01224678   
2  NCT02273973  NCT00281697   
3  NCT00593346          NaN   
4  NCT02340221          NaN   

                                           Statement          Label  \
0  there is a 13.2% difference between the result...  Contradiction   
1  Patients with significantly elevated ejection ...  Contradiction   
2  a significant number of the participants in th...  Contradiction   
3  the primary trial does not report the PFS or o...     Entailment   
4  Prior treatment with fulvest

In [45]:
test_input = encode_data_to_tensors(TEST_HYPOTESIS, TEST_EVIDENCES, tokenizer, VECTOR_MAX)

# check shape data 
print('DEBUG test shapes: ')
print(test_input['input_word_ids'].shape)
print(test_input['input_mask'].shape)
print(test_input['input_type_ids'].shape)

Dataset MAX (dim): 
129
DEBUG test shapes: 
(369, 130)
(369, 130)
(369, 130)


In [46]:
predictions = [np.argmax(i) for i in model.predict(test_input)]
# print([i for i in model.predict(test_input)])

submission = pd.DataFrame()
submission['UUID'] = TEST_UUID
submission['label'] = TEST_LABEL
submission['prediction'] = predictions


RESULTS_FILE_R = PREFIXG+"/results-"+stamp_time()+".json"
RESULTS_FILE = PREFIXG+"/submission-"+stamp_time()+".csv"
submission.to_csv(RESULTS_FILE, index = False)
submission.head()

12/12 [==============================] - 0s 8ms/step


,UUID,label,prediction
0,1adc970c-d433-44d0-aa09-d3834986f7a2,Contradiction,1
1,1adc970c-d433-44d0-aa09-d3834986f7a2,Contradiction,1
2,6b9162d0-0816-46d4-81af-c60028dcc63b,Contradiction,1
3,6b9162d0-0816-46d4-81af-c60028dcc63b,Contradiction,1
4,6b9162d0-0816-46d4-81af-c60028dcc63b,Contradiction,1


In [47]:
# def postProcessing(file):
#     results = {}
#     CT_index = 0
#     try:
#         STM = pd.read_csv(file)
#         # print(STM.head(5))
#         for id in STM['UUID']:
#             prediction = STM['prediction'][CT_index]
#             # print(prediction)
#             CT_index = CT_index + 1
#             if id in results:
#               results[id] = results[id] + ", " +str(prediction)
#             else:
#               results[id] = str(prediction)
#     except Exception as err:
#         print('\n! Error for file')
#         print(err)
#     return results

# rw = postProcessing(RESULTS_FILE)

## Test Task 1 - test.json


In [48]:
def parseEvSg(clinicalTrialId, evidenceSectionId, PREFIX, DEBUG1, DEBUG2):
    evidences = []
    if DEBUG1:
        print("Load premises: " + clinicalTrialId + " " + evidenceSectionId)
    CT_PATH = PREFIX+'CT_data/'+clinicalTrialId+'.json'
    try:
        with open(CT_PATH) as json_file:
            dataCT = json.load(json_file)
        evidences = dataCT[evidenceSectionId]
    except Exception as err:
        print('\n!!! Error for file: ' + clinicalTrialId)
        print(err)
    if DEBUG2:
        print("\n")
    return evidences

In [49]:
TEST_STM = pd.read_csv(TEST_DATA) #, nrows=40


# exit()
TEST_EVIDENCES=[]
TEST_HYPOTESIS=[]
TEST_UUID=[]
CT_index = -1
for id in TEST_STM['Id']:
    evidences = []
    CT_index = CT_index+1
    CTid = TEST_STM['Primary_id'][CT_index]
    hypotesis = normalizeDocument(TEST_STM['Statement'][CT_index], True)
    hypotesisLen = len(hypotesis.split(' '))
    if (TEST_STM['Type'][CT_index] == "Single"):
      evidences = parseEvSg(CTid, TEST_STM['Section_id'][CT_index], PREFIX, DEBUG1, DEBUG2)
      ax = splitEvidence([], "", "[CLS] "+hypotesis+" [SEP] ", evidences, 0, DEBUG2)
      for evidenceFragment in ax:
        # parse only evidences those that have a minimum of words 
        if(hypotesisLen+len(evidenceFragment) > hypotesisLen + 10):
          TEST_EVIDENCES.append(evidenceFragment)
          TEST_HYPOTESIS.append(hypotesis)
          TEST_UUID.append(id)
      if DEBUG1:
          print('Analyzed CT entry: ' + str(CT_index) + "  -  "+ CTid+"\n")
    if (TEST_STM['Type'][CT_index] == "Comparison"):
      CTidSecondary = TEST_STM['Secondary_id'][CT_index]
      premisesPrimary = parseEvSg(CTid, TEST_STM['Section_id'][CT_index], PREFIX, DEBUG1, DEBUG2)
      premisesSecondary = parseEvSg(CTidSecondary, TEST_STM['Section_id'][CT_index], PREFIX, DEBUG1, DEBUG2)
      premisesIndexed = splitEvidence([], "", hypotesis, premisesPrimary + premisesSecondary, 0, DEBUG2)
      if (CT_index < 17):
        print(hypotesis)
      for evidenceFragment in premisesIndexed:
        # parse only evidences those that have a minimum of words 
        if(hypotesisLen+len(evidenceFragment) > hypotesisLen + 10):
          TEST_EVIDENCES.append(evidenceFragment)
          TEST_HYPOTESIS.append(hypotesis)
          TEST_UUID.append(id)
        if (CT_index < 17):
          print("     - "+ evidenceFragment)
      # print("-------2--------")
      if DEBUG1:
          print('Analyzed CT entry: ' + str(CT_index) + "  -  "+ CTid+"\n")


with significantly elevated ejection fraction are excluded from primary  but still be eligible for secondary if are 55 years of age or over
     - Inclusion criteria: Inclusion Criteria:   Female patients age 18 years or older   Histologically proven breast cancer after failure or relapse of no more than three lines of chemotherapy including adjuvant, irrespective of prior hormone therapy metastatic disease (stage IV);   HER2-negative patients (HER2 1+ or negative, or HER2 2+ and FISH negative)   At least one measurable tumour lesion (RECIST);   Availability of tumour samples 
     -   Written informed consent that is consistent with ICH-GCP guidelines and local law  Eastern Cooperative Oncology Group (ECOG, R01-0787) performance score 0 - 2. Exclusion criteria: Exclusion Criteria:   Active infectious disease   Gastrointestinal disorders that may interfere with the absorption of the study drug or chronic diarrhoea   Serious illness, concomitant non-oncological disease or mental problem

In [50]:
validation_input = encode_data_to_tensors(TEST_HYPOTESIS, TEST_EVIDENCES, tokenizer, VECTOR_MAX)

# check shape data 
print('DEBUG test shapes: ')
print(validation_input['input_word_ids'].shape)
print(validation_input['input_mask'].shape)
print(validation_input['input_type_ids'].shape)

Dataset MAX (dim): 
129
DEBUG test shapes: 
(733, 130)
(733, 130)
(733, 130)


In [51]:
predictions = [np.argmax(i) for i in model.predict(validation_input)]
# print([i for i in model.predict(test_input)])

submission = pd.DataFrame()
submission['UUID'] = TEST_UUID
submission['prediction'] = predictions


RESULTS_FILE_R = PREFIXG+"/test-results-"+stamp_time()+".json"
RESULTS_FILE = PREFIXG+"/test-submission-"+stamp_time()+".csv"
submission.to_csv(RESULTS_FILE, index = False)
submission.head()

23/23 [==============================] - 0s 8ms/step


,UUID,prediction
0,1adc970c-d433-44d0-aa09-d3834986f7a2,1
1,1adc970c-d433-44d0-aa09-d3834986f7a2,1
2,6b9162d0-0816-46d4-81af-c60028dcc63b,1
3,6b9162d0-0816-46d4-81af-c60028dcc63b,1
4,6b9162d0-0816-46d4-81af-c60028dcc63b,1


# Postprocessing

Conversion from json to csv of training, test and dev files. 

In [52]:
import json
import pandas as pd


"""
type = train | dev
"""
def preprocessing(JSONPATH, type, PREFIX):

    with open(JSONPATH) as json_file:
        data = json.load(json_file)

    columns = ['Type', 'Section_id', 'Primary_id', 'Secondary_id', 'Statement', 'Label', ] #  'Primary_evidence_index', 'Secondary_evidence_index',

    ids = []
    types = []
    sectionId = []
    primaryId = []
    secondaryId = []
    statement = []
    label = []
    PEI = []
    SEI = []
    for entry in data:
        ids.append(entry)
        types.append(data[entry]['Type'])
        sectionId.append(data[entry]['Section_id'])
        primaryId.append(data[entry]['Primary_id'])
        if not (data[entry].get('Secondary_id') is None):
            secondaryId.append(data[entry]['Secondary_id'])
        else:
            secondaryId.append(None)
        statement.append(data[entry]['Statement'])
        label.append(data[entry]['Label'])
        PEI.append(','.join(str(x) for x in data[entry]['Primary_evidence_index']))
        if not (data[entry].get('Secondary_evidence_index') is None):
            SEI.append(','.join(str(x) for x in data[entry]['Secondary_evidence_index']))
        else:
            SEI.append(','.join(str(x) for x in []))

    df = pd.DataFrame()
    df['Id'] = ids
    df['Type'] = types
    df['Section_id'] = sectionId
    df['Primary_id'] = primaryId
    df['Secondary_id'] = secondaryId
    df['Statement'] = statement
    df['Label'] = label
    df['Primary_evidence_index'] = PEI
    df['Secondary_evidence_index'] = SEI

    #print(df.head(20))

    compression_opts = dict(method='zip', archive_name='out.csv') 
    # df.to_csv('out.zip', index=False, compression=compression_opts) 
    df.to_csv(PREFIX+type+'.csv', index=False) 




# preprocessing("/content/dev.json", "dev", "")

## Metrics (F1 score) script

In [40]:
!pip install sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_distances
import numpy
from sklearn.metrics import f1_score,precision_score,recall_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [41]:
import json
import pandas as pd
from functools import reduce 


dev_path = PREFIXG+"dev.json"
with open(dev_path) as json_file:
    dev = json.load(json_file)


def postProcessing(file):
    results2 = {}
    results = {}
    CT_index = 0
    try:
        STM = pd.read_csv(file)
        # print(STM.head(5))
        for id in STM['UUID']:
            prediction = STM['prediction'][CT_index]
            # print(prediction)
            CT_index = CT_index + 1
            if id in results:
              results[id] = results[id] + "," +str(prediction)
            else:
              results[id] = str(prediction)
    except Exception as err:
        print('\n! Error for file')
        print(err)

    for o in results:
        # print(len(results[o]))
        value = {}
        if(len(results[o])>1):
          nums = results[o].split(',')
          preds = list(map(lambda n: int(n), nums))
          # print(preds)
          lst_len= len(preds)
          lst_avg = reduce(lambda x, y: x + y, preds)/lst_len
          if (lst_avg >= 0.5):
            value['Prediction'] = 'Entailment'
          else:
            value['Prediction'] = 'Contradiction'
        else:
          value['Prediction'] = 'Entailment' if int(results[o]) == 1 else 'Contradiction'
        results2[o] = value
    return results2


w = postProcessing(RESULTS_FILE)


def run_score_script(Results):
    gold = dev
    results = Results  
    uuid_list = list(results.keys())

    results_pred = []
    gold_labels = []
    for i in range(len(uuid_list)):
        if results[uuid_list[i]]["Prediction"] == "Entailment":
            results_pred.append(1)
        else:
            results_pred.append(0)
        if gold[uuid_list[i]]["Label"] == "Entailment":
            gold_labels.append(1)
        else:
            gold_labels.append(0)

    f_score = f1_score(gold_labels,results_pred)
    p_score = precision_score(gold_labels,results_pred)
    r_score = recall_score(gold_labels,results_pred)

    print('F1:{:f}'.format(f_score))
    print('precision_score:{:f}'.format(p_score))
    print('recall_score:{:f}'.format(r_score))


In [42]:
print(w)

{'1adc970c-d433-44d0-aa09-d3834986f7a2': {'Prediction': 'Entailment'}, '6b9162d0-0816-46d4-81af-c60028dcc63b': {'Prediction': 'Entailment'}, '0b6cc8e3-69ee-4a91-b93d-2ad3fddce65f': {'Prediction': 'Entailment'}, 'cc1f712a-2116-4e40-9810-f315e3fa5ff8': {'Prediction': 'Entailment'}, '904061c0-14fa-4f13-9118-9a41e24fa8eb': {'Prediction': 'Entailment'}, '43ee7645-ce1e-42d5-9a74-3e379f6f367b': {'Prediction': 'Entailment'}, '0cef8c8e-7986-46c7-a597-c5733a9899c0': {'Prediction': 'Entailment'}, '43ce26e5-03fa-4e9d-b0eb-6ea356295753': {'Prediction': 'Entailment'}, '3facad41-0221-42f8-834d-470e65c4aad5': {'Prediction': 'Entailment'}, '9cbc00e9-3a2d-4471-a93e-72c95132fb6a': {'Prediction': 'Entailment'}, '8b91cab9-d858-45f3-bf8d-3d6fc55b4818': {'Prediction': 'Entailment'}, '4a75574c-fa86-4e62-a210-81c7b98a3807': {'Prediction': 'Entailment'}, 'd0b50aeb-aad8-4a8d-aae6-5c58a7d382c7': {'Prediction': 'Entailment'}, 'b0b61978-57db-4a1c-812c-509e8b05f2dc': {'Prediction': 'Entailment'}, '24b85b44-b9e6-4c28

Run score script and write results in correct format

In [43]:
run_score_script(w)

with open(RESULTS_FILE_R, 'w') as convert_file:
  convert_file.write(json.dumps(w))

F1:0.668919
precision_score:0.505102
recall_score:0.990000
